## The data figure we want to reproduce (from [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3039263/figure/fig01/))

![data-figure](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3039263/bin/tjp0589-0101-f1.jpg)

In [ ]:
%matplotlib inline

# Built-in libs
import os
import sys
import warnings

# Common libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import quantities as pq

# Open Worm libs
import owtests
import c302

# Testing libs
warnings.simplefilter('ignore')
import sciunit
from neuronunit.tests import waveform,dynamics
import quantities as pq
from c302.muscleunit import vm_plot,models as sciunit_models

In [ ]:
# Create a MuscleModel instance with a particular configuration
# This model has a neuron and a muscle cell
config = 'IClampBWM'
parameter_set = 'C2'
duration = 10000 # ms
dt = 0.05 # ms
model = sciunit_models.MuscleModel(config, parameter_set, duration, dt, config_package="notebooks.configs")

## Action potential waveform shape tests

In [ ]:
# Dictionary of observations, in this case two ephys properties from one paper

doi = 'doi:10.1113/jphysiol.2010.200683' # The DOI of the paper
# Observations copied from the table in the paper
observations={doi:{'ap_amplitude':{'mean':45.1*pq.mV,
                                   'sem':0.7*pq.mV,
                                   'n':25},
                   'ap_width':{'mean':19.7*pq.ms,
                               'sem':1.0*pq.ms,
                               'n':25}}}    

# Instantiate two tests based on these properties
ap_width_test = waveform.APWidthTest(observation=observations[doi]['ap_width'])
ap_amplitude_test = waveform.APAmplitudeTest(observation=observations[doi]['ap_amplitude'])

## Interspike interval distribution tests

In [ ]:
# Load a digitized version of an ISI histogram
histogram_path = os.path.join(owtests.TESTS_HOME,"owtests/CElegansNeuroML/data/digitized_histogram.csv")
df = pd.read_csv(histogram_path,header=None,names=['Bin Centers','Counts'],index_col=0)
df = df.astype('int') # Set ISI counts to integers
df.index = df.index.values.round(3) # Round the bin centers to the nearest ms
df.head()

In [ ]:
width = df.index.values[1] - df.index.values[0] # Bin width from the original histogram
df.plot.bar(align='center',width=1)
plt.title('ISI Histogram');
plt.xlabel('ISI (s)');

In [ ]:
# Make surrogate data for a MLE fit
surrogate_data = []
for bin_center in df.index:
    count = int(df.loc[bin_center])
    surrogate_data += [bin_center]*count

In [ ]:
# Do the fit on the surrogate data to get the estmated parameters
from scipy.stats import gamma
isi_shape,isi_loc,isi_scale = gamma.fit(surrogate_data)
isi_mean, isi_var, isi_skew, isi_kurt = gamma.stats(isi_shape,loc=isi_loc,scale=isi_scale, moments='mvsk')
isi_std = np.sqrt(isi_var)
isi_cv = isi_std/isi_mean
bins = list(df.index - width/2) # Get the left edges of the bins
bins += [bins[-1]+width] # Add the right edge of the last bin
plt.figure(figsize=(10,5))
plt.hist(surrogate_data,bins=bins,normed=True,label='Empirical data')
xs = np.linspace(0,df.index.max(),1000)
plt.plot(xs,gamma.pdf(xs,isi_shape,loc=isi_loc,scale=isi_scale),label='Gamma Distribution Fit')
plt.ylabel('Probability Density')
plt.xlabel('Inter-spike interval (s)')
plt.legend()
print('Gamma parameters:\n\tShape: {:.2g}\n\tScale: {:.2g}\n\tLoc: {:.2g}'.format(isi_shape,isi_scale,isi_loc))
print('Statistics:\n\tMean: {:.2g}\n\tStandard Deviation {:.2g}'.format(isi_mean,isi_std))
plt.savefig('histogram.png',format='png')

In [ ]:
# ISI Mean Test
observation = {doi:
                {'mean':isi_mean*pq.s,
                 'std':isi_std*pq.s,
                 'cv':isi_cv}}
isi_test = dynamics.ISITest(observation=observation[doi])
cv_test = dynamics.ISICVTest(observation=observation[doi])

In [ ]:
# Put them all together in a test suite
ap_tests = sciunit.TestSuite([ap_width_test,ap_amplitude_test,isi_test,cv_test], name="AP ISI and CV Tests")

In [ ]:
# Judge the membrane potential of the muscle 
# One could judge the neuron instead with appropriate keyword parameters, see below)
%time score_matrix = ap_tests.judge(model)
import pandas as pd
score_matrix.style.set_properties(**{'font-size':'16pt'})

In [ ]:
score_matrix

In [ ]:
score_matrix['APAmplitudeTest'][0].observation,score_matrix['APAmplitudeTest'][0].prediction

In [ ]:
score_matrix.T

## Auxiliary data visualization

In [ ]:
# Plot the model output for one of the tests
plt.rcParams.update({'font.size':26,'figure.figsize':(8,6)})
score_matrix['APWidthTest']['LEMS_c302_C2_IClampBWM'].plot_vm()
plt.tight_layout()
plt.savefig('model.png',format='png',dpi=150)

## Model methods could also be run directly as follows

In [ ]:
# These will use the cached results and so be much faster than the original run
%time neuron_vm = model.get_membrane_potential_neuron()
%time muscle_vm = model.get_membrane_potential_muscle()

In [ ]:
# Another way to plot the model output, in this case by cell type
vm_plot(muscle_vm,"Muscle")

In [ ]:
# Same as above, but for a different cell type in this model
# Note that in this flavor of the model, the neuron isn't doing anything at all, 
# and the muscle cell is firing spontaneously
vm_plot(neuron_vm,"Neuron")